In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [1]:
# Read file

words = None
with open('names.txt') as f:
    words = f.read().splitlines()

print(words[:10])

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia', 'harper', 'evelyn']


In [ ]:
# Build vocabulary and encoder/decoder

In [ ]:
# Build dataset (no cross validation)

# ... -> e
# ..e -> m
# .em -> m
# emm -> a
# mma -> .

In [ ]:
# Build the dataset (with dev and test split)

In [ ]:
# Build embeddings table

In [ ]:
# Initialize parameters W1, b1, W2, b2 (with generator) in a params list

In [ ]:
# Check num of elements

In [ ]:
# Do forward and backward propagation

In [ ]:
# Plot the loss and steps

In [ ]:
# Evaluate the training and dev loss

In [ ]:

# Sample from the model

# Notes

Batch size can be adjusted to converge faster and also reduce noise in loss